In [6]:
import numpy as np
import pandas as pd
import os

import sys

sys.path.extend(["../"])

from audio_slowfast.utils.metrics import topk_accuracies_slide, multitask_topk_accuracies_slide

In [7]:
res_path = "../results/res-2s.pkl"
os.path.exists(res_path)

True

In [8]:
df = pd.read_pickle(res_path)

verbs_probs = df["verb_output"]
verbs_labels = df["verb_labels"]
nouns_probs = df["noun_output"]
nouns_labels = df["noun_labels"]

In [9]:
unique_pred_verb = np.argmax(verbs_probs, axis=1)
unique_pred_noun = np.argmax(nouns_probs, axis=1)

assert unique_pred_verb.shape[0] == verbs_probs.shape[0]
assert unique_pred_noun.shape[0] == nouns_probs.shape[0]

In [10]:
np.unique(unique_pred_verb)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 35,
       36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 53, 56, 57, 58,
       60, 67, 68, 69, 70, 72, 74, 77])

## Get results for `WHOLE_VIDEO` mode

In [13]:
action_acc = multitask_topk_accuracies_slide(
    (verbs_probs, nouns_probs),
    (verbs_labels, nouns_labels),
    ks=(1,),
    per_action_instance=False,
)
verb_acc = topk_accuracies_slide(
    verbs_probs,
    verbs_labels,
    ks=(1,),
    per_action_instance=False,
)
noun_acc = topk_accuracies_slide(
    nouns_probs,
    nouns_labels,
    ks=(1,),
    per_action_instance=False,
)

print(f"A: {action_acc:.2f}\tV: {verb_acc:.2f}\tN: {noun_acc:.2f}")

TypeError: 'int' object is not callable

## Get results for `IN_ACTION_BOUNDS` mode

In [57]:
# Find rows where all elements are -1
labels_to_discard_verbs = np.all(verbs_labels == -1, axis=1)
labels_to_discard_nouns = np.all(nouns_labels == -1, axis=1)

assert nouns_labels[~labels_to_discard_nouns].shape == verbs_labels[~labels_to_discard_verbs].shape